In [1]:
#include <lodepng_config_cling.h>
#include <vtkm_config_cling.h>
#include <iostream>
#include <string>
#include <VTKmXeusRender.h>
#include <adios2.h>

#pragma cling load("adios2")

In [2]:
#include <vtkm/cont/testing/MakeTestDataSet.h>
std::unique_ptr<VTKmXeusRender> renderer;


In [3]:

bool running = true;

In [4]:
void display()
{
    std::string filename("/home/mark/Projects/raytracingtherestofyourlife/build/debug/direct.bp");
    /** ADIOS class factory of IO class objects, DebugON is recommended */
    adios2::ADIOS adios(adios2::DebugON);

    /*** IO class object: settings and factory of Settings: Variables,
     * Parameters, Transports, and Execution: Engines */
    adios2::IO bpIO = adios.DeclareIO("ReadBP");

    /** Engine derived class, spawned to start IO operations */
    adios2::Engine bpReader = bpIO.Open(filename, adios2::Mode::Read);

    /** Write variable for buffering */
    adios2::Variable<float> bpFloats =
    bpIO.InquireVariable<float>("pnms");


    if (bpFloats)
    {
        constexpr int nx = 128;
        constexpr int ny = 128;
        
        std::vector<float> depthBuffer;
        std::vector<vtkm::Vec<vtkm::Float32,4>> useful(nx*ny);
        bpReader.Get<float>(bpFloats, depthBuffer, adios2::Mode::Sync);
        std::cout << "Float vector inside " << filename << ": {" << depthBuffer.size() << "}";
        
         for (int i=0; i<useful.size(); i++){
        {
            //std::cout << x << ", ";
            useful[i][0] = depthBuffer[i*4];
            useful[i][1] = depthBuffer[i*4+1];
            useful[i][2] = depthBuffer[i*4+2];
            useful[i][3] = 0;
            
        }
        std::cout << "}\n";
        vtkm::cont::ArrayHandle arr = vtkm::cont::make_ArrayHandle(useful);
        renderer = std::make_unique<VTKmXeusRender>();
        renderer->Display(arr, nx,ny);
    }
}

In [5]:
[](){auto try_and_err = []() {
    
    renderer = std::make_unique<VTKmXeusRender>();
    display();
    
};try_and_err();}();

IncrementalExecutor::executeFunction: symbol '_Z13convertBufferIfEvRSt6vectorIhSaIhEEiiiiRT_' unresolved while linking [cling interface function]!
You are probably missing the definition of void convertBuffer<float>(std::vector<unsigned char, std::allocator<unsigned char> >&, int, int, int, int, float&)
Maybe you need to load the corresponding shared library?
